### Задача 1: JSON Hell

- Сгенерируйте данные и сохраните их в JSON.
- Попробуйте прочитать JSON.
- Теперь сохраните JSON с опцией .option("multiline", "true") (представьте, что одна запись занимает несколько строк). Прочитайте обратно.

Цель: Понять, что JSON — сложный формат, и опции решают всё.

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, ArrayType

spark = SparkSession.builder \
    .appName("Work_with_JSON") \
    .getOrCreate()

df_data = [
    (1, "John", 30, ["Python", "Spark"], {"city": "Moscow", "salary": 100000}),
    (2, "Alice", 25, ["Java", "SQL"], {"city": "SPb", "salary": 90000}),
    (3, "Bob", 35, ["Scala", "Kafka"], {"city": "Moscow", "salary": 120000})
]

data_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("skills", ArrayType(StringType()), True),
    StructField("metadata", StructType([
        StructField("city", StringType(), True),
        StructField("salary", IntegerType(), True)
    ]), True)
])

In [4]:
df = spark.createDataFrame(df_data, schema=data_schema)

path_dir = "/home/jovyan/workspace/data/"
df.write.mode("overwrite").json(path_dir + "example.json")


In [5]:
# чтение json файла
json_df = spark.read \
        .schema(data_schema) \
        .json(path_dir + "example.json")
json_df.show()


+---+-----+---+---------------+----------------+
| id| name|age|         skills|        metadata|
+---+-----+---+---------------+----------------+
|  1| John| 30|[Python, Spark]|{Moscow, 100000}|
|  3|  Bob| 35| [Scala, Kafka]|{Moscow, 120000}|
|  2|Alice| 25|    [Java, SQL]|    {SPb, 90000}|
+---+-----+---+---------------+----------------+



In [6]:
df.write.mode("overwrite") \
        .option("multiline", "true") \
        .json(path_dir + "example.json")
print("Файл успешно сохранен!")

Файл успешно сохранен!


In [8]:
# чтение json файла
json_df_1 = spark.read \
        .schema(data_schema) \
        .json(path_dir + "example.json")
json_df_1.show()

multiline_json = spark.read.option("multiline", "true").json(path_dir + "example.json")
print("Многострочный JSON:")
multiline_json.show()

+---+-----+---+---------------+----------------+
| id| name|age|         skills|        metadata|
+---+-----+---+---------------+----------------+
|  1| John| 30|[Python, Spark]|{Moscow, 100000}|
|  3|  Bob| 35| [Scala, Kafka]|{Moscow, 120000}|
|  2|Alice| 25|    [Java, SQL]|    {SPb, 90000}|
+---+-----+---+---------------+----------------+

Многострочный JSON:
+---+---+----------------+-----+---------------+
|age| id|        metadata| name|         skills|
+---+---+----------------+-----+---------------+
| 30|  1|{Moscow, 100000}| John|[Python, Spark]|
| 35|  3|{Moscow, 120000}|  Bob| [Scala, Kafka]|
| 25|  2|    {SPb, 90000}|Alice|    [Java, SQL]|
+---+---+----------------+-----+---------------+



### Задача 2: Partition Pruning (Оптимизация)

1. Используйте processed_sales (Parquet из практики).
2. Прочитайте его обратно: spark.read.parquet(...).
3. Сделайте фильтр: df.filter("country = 'US'").explain().
4. В выводе explain найдите фразу PartitionFilters.

Если она есть: Поздравляю! Spark понял, что читать папки CN и KR не нужно. Он пошел только в папку US. Это основа оптимизации.

In [10]:
df_parquet = spark.read \
        .format("parquet") \
        .load("/home/jovyan/workspace/data/processed_sales")
df_parquet.show()

+----------+-------+------+-------+
|      date|product|amount|country|
+----------+-------+------+-------+
|2023-01-01|Samsung|   900|     KR|
|2023-01-02| Xiaomi|   400|     CN|
|2023-01-01| iPhone|  1000|     US|
|2023-01-02| iPhone|  1000|     US|
+----------+-------+------+-------+



In [14]:
df_parquet.filter("country = 'US'").explain()

== Physical Plan ==
*(1) ColumnarToRow
+- FileScan parquet [date#186,product#187,amount#188,country#189] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/home/jovyan/workspace/data/processed_sales], PartitionFilters: [isnotnull(country#189), (country#189 = US)], PushedFilters: [], ReadSchema: struct<date:string,product:string,amount:int>


